In [ ]:
import bitstamp.client
import threading
import numpy as np
from collections import defaultdict

In [ ]:
exchange_fee = 0.0025

def print_costs(tickers, print_all=False):
    unique_symbols = set(x['base'] for x in tickers) | set(x['quote'] for x in tickers)
    conversion_rates = defaultdict(dict)
    for ticker in tickers:
        conversion_rates[ticker['quote']][ticker['base']] = {'bid':ticker['bid'], 'ask':ticker['ask']}
        conversion_rates[ticker['base']][ticker['quote']] = {'bid':1.0/ticker['ask'], 'ask':1.0/ticker['bid']}
        
    #BTC is going to be the start node
    graph_symbol_lookup = {'BTC':0}
    for ticker in tickers:
        if ticker['base'] not in graph_symbol_lookup:
            graph_symbol_lookup[ticker['base']] = len(graph_symbol_lookup)
        if ticker['quote'] not in graph_symbol_lookup:
            graph_symbol_lookup[ticker['quote']] = len(graph_symbol_lookup)
    graph_symbol_reverse_lookup = {y:x for x, y in graph_symbol_lookup.items()}

    #Using a dijkstra like approch would be faster but Bitstamp has very few symbols so might as well start simple 
    for i in range(0, len(unique_symbols)):
        for j in range(0, len(unique_symbols)):
            for k in range(0, len(unique_symbols)):
            
                if len(set([i, j, k])) != 3:
                    continue
                    
                c1 = graph_symbol_reverse_lookup[i]
                c2 = graph_symbol_reverse_lookup[j]
                c3 = graph_symbol_reverse_lookup[k]

                #Assumming that we are trading one coin
                initial_equity = current_equity = 1
                if c2 not in conversion_rates[c1] or c3 not in conversion_rates[c2] or c1 not in conversion_rates[c3]:
                    #No conversion is avalible 
                    #print('No conversion for {} -> {} -> {} -> {}'.format(c1, c2, c3, c1))
                    continue 

                current_equity = current_equity * conversion_rates[c1][c2]['ask'] - current_equity*exchange_fee
                current_equity = current_equity * conversion_rates[c2][c3]['ask'] - current_equity*exchange_fee
                current_equity = current_equity * conversion_rates[c3][c1]['ask'] - current_equity*exchange_fee

                profit_loss = current_equity - initial_equity

                if print_all or profit_loss >= 0:
                    print('{} -> {} -> {} -> {} profit/loss = {} {}'.format(c1, c2, c3, c1, c1, profit_loss))


In [ ]:
public_client = bitstamp.client.Public()
pairs = public_client.trading_pairs_info()

while True:
    try:
        tickers = []

        def get_ticker(base, quote):
            ticker = public_client.ticker(base=base, quote=quote)
            ticker['base'] = base
            ticker['quote'] = quote

            ticker['ask'] = float(ticker['ask'])
            ticker['bid'] = float(ticker['bid'])

            tickers.append(ticker)

        threads = []

        for pair in pairs:
            base, quote = pair['name'].split('/')
            thread = threading.Thread(target=get_ticker, args=(base, quote,))
            thread.start()
            threads.append(thread)



        for thread in threads:
            thread.join()


        print_costs(tickers, print_all=True)
    except Exception as ex:
        print(ex)
        public_client = bitstamp.client.Public()